## Latency Check

Feedback : shins777@gmail.com. 

* 이 Colab은 Google model 들의 latency를 확인하기 위한 테스트 입니다.

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform 

Note: you may need to restart the kernel to use updated packages.


### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [2]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  


In [6]:
PROJECT_ID='ai-hangsik'
LOCATION = 'us-central1'

In [8]:
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)
prompt = "생성형 AI가 사회적으로 문제가 될 수 있는 사항들을 알려주세요. "

text_latency = {}
text_response = {}

#### PaLM models

In [ ]:
import time
from vertexai.language_models import TextGenerationModel

palm_model_lists = ['text-bison@001','text-bison@002','text-bison-32k@002','text-unicorn@001']


for model_str in palm_model_lists:

  t1 = time.time()

  parameters = {
      "temperature":0.2,
      "max_output_tokens":1024,
      "top_p":0.8,
      "top_k":10
  }

  model = TextGenerationModel.from_pretrained(model_str)
  response = model.predict(prompt, **parameters)

  t2 = time.time()

  text_response[model_str] = response.text
  text_latency[model_str] = round((t2-t1),2)

#### Gemini Models

In [ ]:
import time
from vertexai.preview.generative_models import GenerativeModel

gemini_model_lists = ["gemini-1.0-pro-001","gemini-1.0-ultra-001"]


for model_str in gemini_model_lists:


  model = GenerativeModel(model_str)

  t1 = time.time()

  responses = model.generate_content(
      prompt,
      generation_config={
          "temperature": 0.2,
          "max_output_tokens": 1024,
          "top_p": 0.8,
          "top_k": 10,
      },
      stream=False
  )

  t2 = time.time()

  text_response[model_str] = responses.text
  text_latency[model_str] = round((t2-t1),2)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

response_df = pd.DataFrame.from_dict(text_response, orient='index', columns=['Response'])
response_df

In [ ]:
latency_df = pd.DataFrame.from_dict(text_latency, orient='index', columns=['Latency'])
latency_df.sort_values(by='Latency', ascending=False, inplace=True)
latency_df

In [ ]:

ax = latency_df.plot(y='Latency', kind='barh', legend=True, rot=0 )
ax.bar_label(ax.containers[0], label_type='edge')
plt.tight_layout()